In [1]:
# Test fetch_initial_data

TOKEN_ADDRESS = "hTRDn7zE5tDHRnjj6Qms2WG1zEGv9ii6AiwfgbFpump"
DEXSCREENER_API = "https://api.dexscreener.com/latest/dex/tokens"
SOLSCAN_API = "https://public-api.solscan.io/token"  # Solana blockchain data
HELIUS_RPC_URL = "https://mainnet.helius-rpc.com/?api-key=cde2166e-a9cc-4f20-aab6-931319852b4a"
TELEGRAM_BOT_TOKEN = "7631318467:AAEgC0za6dp_kRIDLauwp4nz-ur5vT-Lpwc"
TELEGRAM_CHAT_ID = "-1002421555551"

In [3]:
import requests


def fetch_initial_data():
    # Fetch price and liquidity from DexScreener
    dex_data = requests.get(f"{DEXSCREENER_API}/{TOKEN_ADDRESS}").json()
    price = float(dex_data["pairs"][0]["priceUsd"])
    liquidity = float(dex_data["pairs"][0]["liquidity"]["usd"])

    # Get token metadata using Helius RPC
    helius_payload = {
        "jsonrpc": "2.0",
        "id": "my-id",
        "method": "getTokenSupply",
        "params": [TOKEN_ADDRESS]
    }
    
    supply_response = requests.post(
        HELIUS_RPC_URL,
        headers={"Content-Type": "application/json"},
        json=helius_payload
    ).json()
    
    total_supply = float(supply_response["result"]["value"]["amount"])
    
    # Get mint authority using getAccountInfo
    account_payload = {
        "jsonrpc": "2.0",
        "id": "my-id",
        "method": "getAccountInfo",
        "params": [TOKEN_ADDRESS, {"encoding": "jsonParsed"}]
    }
    
    account_response = requests.post(
        HELIUS_RPC_URL,
        headers={"Content-Type": "application/json"},
        json=account_payload
    ).json()
    
    creator = account_response["result"]["value"]["data"]["parsed"]["info"]["mintAuthority"]

    # Calculate MCap
    market_cap = price * total_supply

    return {
        "price": price,
        "liquidity": liquidity,
        "total_supply": total_supply,
        "market_cap": market_cap,
        "creator": creator,
    }

if __name__ == "__main__":
    initial_data = fetch_initial_data()
    print(initial_data)

{'price': 0.003441, 'liquidity': 290846.3, 'total_supply': 999617105499077.0, 'market_cap': 3439682460022.324, 'creator': None}


In [7]:
import requests
import time


# Monitor price changes
def monitor_price(initial_price):
    while True:
        dex_data = requests.get(f"{DEXSCREENER_API}/{TOKEN_ADDRESS}").json()
        current_price = float(dex_data["pairs"][0]["priceUsd"])
        price_change = abs((current_price - initial_price) / initial_price) * 100

        if price_change > 10:  # 10% threshold
            send_alert(f"🚨 Price Alert: {price_change:.2f}% change! Current price: ${current_price}")

        time.sleep(300)  # Check every 5 minutes

if __name__ == "__main__":
    initial_price = float(requests.get(f"{DEXSCREENER_API}/{TOKEN_ADDRESS}").json()["pairs"][0]["priceUsd"])
    monitor_price(initial_price)

KeyboardInterrupt: 

In [8]:
# Monitor liquidity changes
def monitor_liquidity(initial_liquidity):
    while True:
        dex_data = requests.get(f"{DEXSCREENER_API}/{TOKEN_ADDRESS}").json()
        current_liquidity = float(dex_data["pairs"][0]["liquidity"]["usd"])
        liquidity_change = abs((current_liquidity - initial_liquidity) / initial_liquidity) * 100

        if liquidity_change > 5:  # 5% threshold
            send_alert(f"🚨 Liquidity Alert: {liquidity_change:.2f}% change! Current liquidity: ${current_liquidity}")

        time.sleep(600)  # Check every 10 minutes

if __name__ == "__main__":
    initial_liquidity = requests.get(f"{DEXSCREENER_API}/{TOKEN_ADDRESS}").json()["pairs"][0]["liquidity"]["usd"]
    monitor_liquidity(initial_liquidity)

KeyboardInterrupt: 

In [13]:
def monitor_market_cap(initial_price, total_supply):
    while True:
        # Get current price from DexScreener
        dex_data = requests.get(f"{DEXSCREENER_API}/{TOKEN_ADDRESS}").json()
        current_price = float(dex_data["pairs"][0]["priceUsd"])
        
        # Calculate market caps
        initial_market_cap = initial_price * total_supply
        current_market_cap = current_price * total_supply
        
        # Calculate percentage change
        market_cap_change = abs((current_market_cap - initial_market_cap) / initial_market_cap) * 100
        
        if market_cap_change > 7:  # 7% threshold
            send_alert(f"📊 Market Cap Alert:\n" 
                      f"Change: {market_cap_change:.2f}%\n"
                      f"Current MC: ${current_market_cap:,.2f}\n"
                      f"Current Price: ${current_price:,.6f}")
            
        time.sleep(300)  # Check every 5 minutes

if __name__ == "__main__":
    INITIAL_PRICE = initial_price
    TOTAL_SUPPLY = total_supply

    monitor_market_cap(INITIAL_PRICE, TOTAL_SUPPLY)

NameError: name 'total_supply' is not defined

In [14]:
def get_token_creator(token_address):
    """Get token creator using Helius getAsset"""
    payload = {
        "jsonrpc": "2.0",
        "id": "token-info",
        "method": "getAsset",
        "params": {
            "id": token_address
        }
    }
    
    response = requests.post(
        HELIUS_RPC_URL,
        headers={"Content-Type": "application/json"},
        json=payload
    ).json()
    
    if "result" in response:
        authorities = response["result"].get("authorities", [])
        if authorities:
            return authorities[0].get("address")  # Return first authority address
    
    return None

get_token_creator("hTRDn7zE5tDHRnjj6Qms2WG1zEGv9ii6AiwfgbFpump")

'TSLvdd1pWpHVjahSpsvCXUbgwsL3JAcvokwaKt1eokM'

In [17]:
def get_creator_tokens(creator_address):
    """Get all tokens created by address that are listed on Raydium with extended metrics"""
    
    search_payload = {
        "jsonrpc": "2.0",
        "id": "creator-tokens",
        "method": "searchAssets",
        "params": {
            "authorityAddress": creator_address,
            "tokenType": "fungible",
            "page": 1,
            "limit": 100,
            "options": {
                "showNativeBalance": True,
                "showCollectionMetadata": True
            }
        }
    }

    tokens_response = requests.post(
        HELIUS_RPC_URL,
        headers={"Content-Type": "application/json"},
        json=search_payload
    ).json()

    raydium_tokens = []
    if "result" in tokens_response and "items" in tokens_response["result"]:
        for item in tokens_response["result"]["items"]:
            token_address = item["id"]

        # Get token metadata including freeze authority
        token_info_payload = {
            "jsonrpc": "2.0",
            "id": "token-info",
            "method": "getAccountInfo",
            "params": [token_address, {"encoding": "jsonParsed"}],
        }

        token_info = requests.post(
            HELIUS_RPC_URL,
            headers={"Content-Type": "application/json"},
            json=token_info_payload,
        ).json()

        freeze_authority = token_info["result"]["value"]["data"]["parsed"]["info"].get(
            "freezeAuthority"
        )

        # Get top holders data
        holders_payload = {
            "jsonrpc": "2.0",
            "id": "token-holders",
            "method": "getProgramAccounts",
            "params": [
                "TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA",
                {
                    "filters": [
                        {"dataSize": 165},
                        {"memcmp": {"offset": 0, "bytes": token_address}},
                    ],
                    "encoding": "jsonParsed",
                },
            ],
        }

        holders = requests.post(
            HELIUS_RPC_URL,
            headers={"Content-Type": "application/json"},
            json=holders_payload,
        ).json()

        # Calculate top holders percentage
        total_supply = float(
            token_info["result"]["value"]["data"]["parsed"]["info"]["supply"]
        )
        sorted_holders = sorted(
            holders["result"],
            key=lambda x: float(
                x["account"]["data"]["parsed"]["info"]["tokenAmount"]["amount"]
            ),
            reverse=True,
        )
        top_10_percentage = (
            sum(
                float(h["account"]["data"]["parsed"]["info"]["tokenAmount"]["amount"])
                for h in sorted_holders[:10]
            )
            / total_supply
            * 100
        )

        # Get DEX data including LP info
        dex_data = requests.get(f"{DEXSCREENER_API}/{token_address}").json()

        if "pairs" in dex_data:
            raydium_pairs = [
                pair for pair in dex_data["pairs"] if "raydium" in pair["dexId"].lower()
            ]
            if raydium_pairs:
                # Check LP token burn
                lp_address = raydium_pairs[0].get("pairAddress")
                lp_burned = "Yes" if check_lp_burn(lp_address) else "No"

                raydium_tokens.append(
                    {
                        "address": token_address,
                        "name": dex_data["pairs"][0].get("baseToken", {}).get("name"),
                        "symbol": dex_data["pairs"][0].get("baseToken", {}).get("symbol"),
                        "price": dex_data["pairs"][0].get("priceUsd"),
                        "liquidity": dex_data["pairs"][0].get("liquidity", {}).get("usd"),
                        "volume_24h": dex_data["pairs"][0].get("volume", {}).get("h24"),
                        "freeze_authority": freeze_authority,
                        "lp_burned": lp_burned,
                        "top_10_holders_pct": top_10_percentage,
                    }
                )

    return raydium_tokens

In [19]:
raydium_tokens = get_creator_tokens("TSLvdd1pWpHVjahSpsvCXUbgwsL3JAcvokwaKt1eokM")
raydium_tokens

[]